In [1]:
# python warning off
import warnings
warnings.filterwarnings('ignore')

# tensorflow warning off
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import keras

In [2]:
df_full = pd.read_csv("RNNtest.csv")
df_full.head()

,도루_D,도루_R,방어율,자책,완투,실점,득점,타수,안타수,2루,3루,홈런,볼넷,삼진,이닝당아웃수,안타허용,홈런허용,볼넷허용,탈삼진,탈병살,실책,OPS,WHIP,W
0,0,2,3.000000,4,0,4,5,42,9,2,0,1,4,10,36,13,0,3,10,1,0,0.615942,1.333333,1
1,2,0,3.176471,4,0,5,4,46,13,2,0,0,3,11,34,9,1,4,11,0,2,0.652618,1.147059,0
2,0,0,4.000000,4,0,4,5,30,9,2,0,2,5,10,27,8,1,3,10,1,1,0.966667,1.222222,1
3,0,0,5.625000,5,0,5,4,34,8,3,0,1,3,9,24,9,2,5,9,1,0,0.727554,1.750000,0
4,2,0,8.000000,8,0,8,4,36,10,5,0,1,0,7,27,11,2,4,7,0,1,0.777778,1.666667,0


In [3]:
from sklearn.preprocessing import MinMaxScaler

In [4]:
scaler = MinMaxScaler()

In [5]:
scaled_col = ['도루_D', '도루_R', '방어율', '자책', '완투', '실점', '득점', '타수', '안타수', '2루', '3루',
       '홈런', '볼넷', '삼진', '이닝당아웃수', '안타허용', '홈런허용', '볼넷허용', '탈삼진', '탈병살', '실책',
       'OPS', 'WHIP', 'W']

scaled_arr = scaler.fit_transform(df_full[scaled_col])
df_scaled = pd.DataFrame(scaled_arr, columns=scaled_col)

In [6]:
df_scaled

,도루_D,도루_R,방어율,자책,완투,실점,득점,타수,안타수,2루,3루,홈런,볼넷,삼진,이닝당아웃수,안타허용,홈런허용,볼넷허용,탈삼진,탈병살,실책,OPS,WHIP,W
0,0.000000,0.285714,0.151515,0.210526,0.0,0.210526,0.263158,0.722222,0.346154,0.222222,0.0,0.166667,0.307692,0.588235,1.000000,0.500000,0.000000,0.230769,0.588235,0.25,0.000000,0.347644,0.303797,1.0
1,0.285714,0.000000,0.160428,0.210526,0.0,0.263158,0.210526,0.833333,0.500000,0.222222,0.0,0.000000,0.230769,0.647059,0.904762,0.346154,0.166667,0.307692,0.647059,0.00,0.333333,0.373830,0.247208,0.0
2,0.000000,0.000000,0.202020,0.210526,0.0,0.210526,0.263158,0.388889,0.346154,0.222222,0.0,0.333333,0.384615,0.588235,0.571429,0.307692,0.166667,0.230769,0.588235,0.25,0.166667,0.598055,0.270042,1.0
3,0.000000,0.000000,0.284091,0.263158,0.0,0.263158,0.210526,0.500000,0.307692,0.333333,0.0,0.166667,0.230769,0.529412,0.428571,0.346154,0.333333,0.384615,0.529412,0.25,0.000000,0.427333,0.430380,0.0
4,0.285714,0.000000,0.404040,0.421053,0.0,0.421053,0.210526,0.555556,0.384615,0.555556,0.0,0.166667,0.000000,0.411765,0.571429,0.423077,0.333333,0.307692,0.411765,0.00,0.166667,0.463192,0.405063,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,0.000000,0.142857,0.340909,0.315789,0.0,0.368421,0.315789,0.555556,0.384615,0.222222,0.0,0.333333,0.230769,0.529412,0.428571,0.461538,0.500000,0.230769,0.529412,0.25,0.333333,0.502858,0.468354,0.0
1436,0.142857,0.571429,0.353535,0.368421,0.0,0.368421,0.210526,0.638889,0.461538,0.444444,0.0,0.000000,0.076923,0.411765,0.571429,0.538462,0.000000,0.230769,0.411765,0.00,0.000000,0.444587,0.472574,0.0
1437,0.571429,0.142857,0.151515,0.157895,0.0,0.210526,0.368421,0.611111,0.538462,0.333333,0.0,0.000000,0.230769,0.764706,0.571429,0.461538,0.000000,0.076923,0.764706,0.00,0.166667,0.516278,0.337553,1.0
1438,0.000000,0.142857,0.252525,0.263158,0.0,0.263158,0.421053,0.555556,0.423077,0.222222,0.0,0.333333,0.076923,0.352941,0.571429,0.384615,0.000000,0.230769,0.352941,0.25,0.000000,0.528953,0.337553,1.0


In [7]:
TEST_SIZE = 100

train = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]

In [8]:
def make_dataset(data, label, window_size=10):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [9]:
feature_cols = ['도루_D', '도루_R', '방어율', '자책', '완투', '실점', '득점', '타수', '안타수', '2루', '3루',
       '홈런', '볼넷', '삼진', '이닝당아웃수', '안타허용', '홈런허용', '볼넷허용', '탈삼진', '탈병살', '실책',
       'OPS', 'WHIP']
label_cols = ['W']

In [10]:
train_feature, train_label = make_dataset(train[feature_cols], train[label_cols],
                                         10)

In [11]:
train_feature.shape, train_label.shape

((1330, 10, 23), (1330, 1))

In [12]:
test_feature, test_label = make_dataset(test[feature_cols], 
                                        test[label_cols], 10)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(train_feature, train_label,
                                                      test_size=0.2)

In [15]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense
from keras.layers import Dropout

from keras.backend import clear_session

In [21]:
def build_model(n_hidden=1, n_neurons=16, input_shape=[10,23]):
    model = Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        if n_hidden == 1:
            model.add(keras.layers.LSTM(n_neurons, activation="selu"))
        else:
            model.add(keras.layers.LSTM(n_neurons, activation="selu", return_sequence=True))
            if layer == n_hidden - 1:
                model.add(keras.layers.LSTM(n_neurons, activation="selu"))
    model.add(keras.layers.Dense(2, activation='softmax'))
    model.compile(loss="sparse_categorical_crossentropy", 
                  optimizer='adam', metrics=['accuracy'])
    return model

In [22]:
keras_LSTM = build_model()

In [23]:
keras_LSTM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 16)                2560      
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 34        
Total params: 2,594
Trainable params: 2,594
Non-trainable params: 0
_________________________________________________________________


In [25]:
early_stop

keras_LSTM.fit(X_train, y_train, epochs=200, batch_size=15,
          validation_data=(X_valid, y_valid),
          callbacks=EarlyStopping(monitor='val_loss', patience=10))

Train on 1064 samples, validate on 266 samples


TypeError: unsupported operand type(s) for +: 'EarlyStopping' and 'list'

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV